In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from qiskit import *
from qiskit_machine_learning.algorithms.classifiers import VQC
from qiskit.circuit.library import EfficientSU2, TwoLocal, NLocal, RealAmplitudes, ZFeatureMap, ZZFeatureMap, PauliFeatureMap
from qiskit.circuit.library import CCXGate, CRZGate, RXGate
from qiskit.algorithms.optimizers import COBYLA, SLSQP, SPSA
from matplotlib import pyplot as plt
from IPython.display import clear_output
from qiskit.primitives import Sampler
from qiskit.circuit import ParameterVector
from sklearn.metrics import recall_score, precision_score, f1_score
import time
from qiskit_machine_learning.algorithms.classifiers import VQC
import os

In [5]:
train = pd.read_csv("../data/train_fe_small.csv")
test = pd.read_csv("../data/test_fe.csv")

In [6]:
cols = ['region_South', 'region_West', 'account_length','number_vmail_messages', 'total_day_minutes', 'total_day_calls',
        'total_intl_charge', 'customer_service_calls', 'churn']

In [7]:
train = train[cols]

In [8]:
x_train_use, y_train_use = train.drop("churn", axis = 1), train["churn"]

In [9]:
test = test[cols]

In [10]:
x_test, y_test = test.drop("churn", axis =1), test["churn"]

In [21]:
pauli_feature_map = PauliFeatureMap(feature_dimension=len(x_train_use.columns), reps=1)
sampler = Sampler()
ansatz_su = EfficientSU2(num_qubits=pauli_feature_map.width(), reps = 2, su2_gates=["ry", "rz"], entanglement= "full",
                         insert_barriers=True)
num_iter=300
cobyla = COBYLA(maxiter = num_iter)
model = VQC(
        sampler=sampler,
        feature_map=pauli_feature_map,
        ansatz=ansatz_su,
        optimizer=cobyla)

In [18]:
model.load("../vqc_model/train_process/su2_cobyla.model")

In [14]:
x_train_arr = x_train_use.to_numpy()
y_train_arr = y_train_use.to_numpy()
x_test_arr = x_test.to_numpy()
y_test_arr = y_test.to_numpy()

In [16]:
model.fit(x_train_arr, y_train_arr)

/home/ganiyuma/.local/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
capi_return is NULL
Call-back cb_calcfc_in__cobyla__user__routines failed.


KeyboardInterrupt: 

In [19]:
pred_use = model.predict(x_train_arr)

QiskitMachineLearningError: 'The model has not been fitted yet'

In [ ]:
pred_test = model.predict(x_test_arr)

In [ ]:
f1_train = f1_score(y_train_use, pred_use)
prec_train = precision_score(y_train_use, pred_use)
recall_train = recall_score(y_train_use, pred_use)

In [ ]:
f1_test = f1_score(y_test, pred_test)
prec_test = precision_score(y_test, pred_test)
recall_test = recall_score(y_test, pred_test)

In [ ]:
df = pd.DataFrame()
df["f1_test"] = [f1_test]
df["f1_train"] = f1_train
df["prec_train"] = prec_train
df["prec_test"] = prec_test
df["recall_train"] = recall_train
df["recall_test"] = recall_test
df["model"] = "VQC_best"

In [ ]:
df.to_csv("../vqc_results/final/vqc.csv", index=False)